TITLE ARTICLE:

In [2]:
%pip install elasticsearch openai pydantic langchain-openai -q


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import base64
import json
import os
import uuid
from getpass import getpass
from typing import Any, Dict, List, Literal, Optional

from elasticsearch import Elasticsearch
from langchain_openai import ChatOpenAI
from openai import OpenAI
from pydantic import BaseModel, Field

In [4]:
INDEX_NAME = "kibana_sample_data_logs"
IMAGE_PATH = "imgs/dashboard.png"

os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
os.environ["ELASTICSEARCH_API_KEY"] = getpass("Enter your Elasticsearch API key: ")
os.environ["ELASTICSEARCH_URL"] = getpass("Enter your Elasticsearch URL: ")
os.environ["KIBANA_URL"] = getpass("Enter your Kibana URL: ")

In [ ]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

langchain_llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

es_client = Elasticsearch(
    [os.getenv("ELASTICSEARCH_URL")],
    api_key=os.getenv("ELASTICSEARCH_API_KEY"),
)


class Visualization(BaseModel):
    title: str = Field(description="The dashboard title")
    type: List[Literal["pie", "bar", "metric"]]
    field: Optional[str] = Field(
        description="The field that this visualization use based on the provided mappings"
    )


class Dashboard(BaseModel):
    title: str = Field(description="The dashboard title")
    visualizations: List[Visualization]

### Loading the json templates

There are 3 templates for each visualization type:
- pie
- bar
- metric

The templates are in the templates folder.

The templates are in the following format:
- insBar.json
- insPie.json
- insMetric.json


In [6]:
templates = {}
template_dir = os.path.join(os.path.dirname(__file__), "templates")

for vis_type in ["pie", "bar", "metric"]:
    template_file = os.path.join(template_dir, f"lns{vis_type.capitalize()}.json")

    try:
        with open(template_file, "r") as f:
            templates[vis_type] = json.load(f)
    except FileNotFoundError:
        print(f"Warning: Template file {template_file} not found")
        templates[vis_type] = {}

Function to get templates by visualization type:



In [ ]:
def get_chart_template(vis_type: str):
    return templates.get(vis_type, templates.get("bar", {}))

Function to insert the values generated by the LLM into the template:

In [ ]:
def fill_template_with_analysis(
    template: Dict[str, Any],
    visualization: Visualization,
):
    template_str = json.dumps(template)
    replacements = {
        "{title}": visualization.title,
    }
    if visualization.field:
        replacements["{field}"] = visualization.field
    for placeholder, value in replacements.items():
        template_str = template_str.replace(placeholder, str(value))

    return json.loads(template_str)

Retrieve index mappings for the index that the dashboard is based on.

In [ ]:
result = es_client.indices.get_mapping(index=INDEX_NAME)
index_mappings = result[list(result.keys())[0]]["mappings"]["properties"]

Loading image:

In [ ]:
image_base64 = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("utf-8")

In [ ]:
dashboard_schema = json.dumps(Dashboard.model_json_schema(), indent=2)

prompt = f"""
    You are an expert in analyzing Kibana dashboards from images for the version 9.0.0 of Kibana.

    You will be given a dashboard image and a Elasticsearch index mappings.

    Below is the index mappings for the index that the dashboard is based on.
    Use this to help you understand the data and the fields that are available.

    Index Mappings:
    {index_mappings}

    Return a JSON object with the following structure (see JSON schema below):
    {dashboard_schema}
    
    There are some things to consider:
    - field: The field that is relevant for the visualization, this is the data that will be used to generate the chart and can be found in the index mappings.
    - If you choose a field that has multi-field, you can use the field name with the suffix .keyword to get the multi-field. For example: 
        'field_name': {{
            'type': 'text',
            'fields': {{
                'keyword': {{
                    'type': 'keyword',
                }}
            }}
        }}
        Here you can use the field_name.keyword to get the multi-field.
        
        'field_name': {{
            'type': 'keyword',
        }}
        Here you can use the field_name to get the single field.
    
    - Only include the fields that are relevant for each visualization, based on what is visible in the image. The output must be a JSON object matching the schema above.
    """

try:
    response = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{image_base64}"},
                    },
                ],
            }
        ],
    )

    content = response.choices[0].message.content
    result = json.loads(content)
    dashboard_values = Dashboard(**result)

except Exception as e:
    print(f"Failed to analyze image and match fields: {str(e)}")

In [ ]:
panels = []
for vis in dashboard_values.visualizations:
    for vis_type in vis.type:
        template = get_chart_template(vis_type)
        filled_panel = fill_template_with_analysis(template, vis)
        panels.append(filled_panel)

### Generate the dashboard

Here is called the API /api/generate-dashboard. The templates with the values generated by the LLM are sent to the API.

In [ ]:
try:
    dashboard_id = str(uuid.uuid4())

    # post request to create the dashboard endpoint
    url = f"{os.getenv('KIBANA_URL')}/api/dashboards/dashboard/{dashboard_id}"

    dashboard_config = {
        "attributes": {
            "title": "Generated Dashboard by LLM",
            "description": "Generated by AI",
            "timeRestore": False,
            "panels": panels,  # Visualizations with the values generated by the LLM
            "timeFrom": "now-7d/d",
            "timeTo": "now",
        },
    }

    headers = {
        "Content-Type": "application/json",
        "kbn-xsrf": "true",
        "Authorization": f"ApiKey {os.getenv('ELASTICSEARCH_API_KEY')}",
    }

    requests.post(
        url,
        headers=headers,
        json=dashboard_config,
    )

    # Url to the generated dashboard
    dashboard_url = f"{os.getenv('KIBANA_URL')}/app/dashboards#/view/{dashboard_id}"

    print("Dashboard URL: ", dashboard_url)
    print("Dashboard ID: ", dashboard_id)

except Exception as e:
    print(f"Failed to create dashboard: {str(e)}")